# Shear instability of a free-surface flow

This script simulates the instability of a sheared, free-surface
flow using `ClimateMachine.Ocean.HydrostaticBoussinesqSuperModel`.

In [1]:
using Printf
using Plots
using ClimateMachine

ClimateMachine.init()

ClimateMachine.Settings.array_type = Array

using ClimateMachine.Ocean
using ClimateMachine.Ocean.Domains
using ClimateMachine.Ocean.Fields

using ClimateMachine.GenericCallbacks: EveryXSimulationTime
using ClimateMachine.Ocean: current_step, Δt, current_time
using CLIMAParameters: AbstractEarthParameterSet, Planet

We begin by specifying the domain and mesh,

In [2]:
domain = RectangularDomain(
    Ne = (24, 24, 1),
    Np = 4,
    x = (-3π, 3π),
    y = (-3π, 3π),
    z = (0, 1),
    periodicity = (true, false, false),
)

RectangularDomain{Float64}
    Np = 4, Ne = (x = 24, y = 24, z = 1)
    L = (x = 18.84955592153876, y = 18.84955592153876, z = 1.0)


Note that the default solid-wall boundary conditions are free-slip and
insulating on tracers. Next, we specify model parameters and the sheared
initial conditions

In [3]:
struct NonDimensionalParameters <: AbstractEarthParameterSet end
Planet.grav(::NonDimensionalParameters) = 1

initial_conditions = InitialConditions(
    u = (x, y, z) -> tanh(y) + 0.1 * cos(x / 3) + 0.01 * randn(),
    v = (x, y, z) -> 0.1 * sin(y / 3),
    θ = (x, y, z) -> x,
)

model = Ocean.HydrostaticBoussinesqSuperModel(
    domain = domain,
    time_step = 0.05,
    initial_conditions = initial_conditions,
    parameters = NonDimensionalParameters(),
    turbulence_closure = (νʰ = 1e-2, κʰ = 1e-2, νᶻ = 1e-2, κᶻ = 1e-2),
    rusanov_wave_speeds = (cʰ = 0.1, cᶻ = 1),
    boundary_tags = ((0, 0), (1, 1), (1, 2)),
    boundary_conditions = (
        OceanBC(Impenetrable(FreeSlip()), Insulating()),
        OceanBC(Penetrable(FreeSlip()), Insulating()),
    ),
);
nothing

┌ Info: Initializing 
└ @ ClimateMachine /central/scratch/climaci/climatemachine-docs/1125/climatemachine-docs/src/Driver/solver_configs.jl:169


We prepare a callback that periodically fetches the horizontal velocity and
tracer concentration for later animation,

In [4]:
u, v, η, θ = model.fields
fetched_states = []

start_time = time_ns()

data_fetcher = EveryXSimulationTime(1) do
    step = @sprintf("Step: %d", current_step(model))
    time = @sprintf("time: %.2f min", current_time(model) / 60)
    max_u = @sprintf("max|u|: %.6f", maximum(abs, u))

    elapsed = (time_ns() - start_time) * 1e-9
    wall_time = @sprintf("elapsed wall time: %.2f min", elapsed / 60)

    @info "$step, $time, $max_u, $wall_time"

    push!(
        fetched_states,
        (u = assemble(u), θ = assemble(θ), time = current_time(model)),
    )
end

ClimateMachine.GenericCallbacks.EveryXSimulationTime(Main.##597.var"#7#8"(), 1, 0)

and then run the simulation.

In [5]:
model.solver_configuration.timeend = 100.0

result = ClimateMachine.invoke!(
    model.solver_configuration;
    user_callbacks = [data_fetcher],
)

┌ Info: Starting 
│     dt              = 5.00000e-02
│     timeend         =   100.00
│     number of steps = 1
│     norm(Q)         = 1.0412196186830819e+02
└ @ ClimateMachine /central/scratch/climaci/climatemachine-docs/1125/climatemachine-docs/src/Driver/Driver.jl:783
┌ Info: Step: 20, time: 0.02 min, max|u|: 1.105455, elapsed wall time: 0.49 min
└ @ Main.##597 string:14
┌ Info: Step: 40, time: 0.03 min, max|u|: 1.084746, elapsed wall time: 0.53 min
└ @ Main.##597 string:14
┌ Info: Step: 61, time: 0.05 min, max|u|: 1.061249, elapsed wall time: 0.58 min
└ @ Main.##597 string:14
┌ Info: Step: 82, time: 0.07 min, max|u|: 1.051660, elapsed wall time: 0.62 min
└ @ Main.##597 string:14
┌ Info: Step: 103, time: 0.09 min, max|u|: 1.061375, elapsed wall time: 0.67 min
└ @ Main.##597 string:14
┌ Info: Step: 124, time: 0.10 min, max|u|: 1.082005, elapsed wall time: 0.72 min
└ @ Main.##597 string:14
┌ Info: Step: 145, time: 0.12 min, max|u|: 1.096369, elapsed wall time: 0.76 min
└ @ Main.##59

0.543863019180897

Finally, we make an animation of the evolving shear instability.

In [6]:
animation = @animate for (i, state) in enumerate(fetched_states)
    local u
    local θ

    @info "Plotting frame $i of $(length(fetched_states))..."

    kwargs =
        (xlim = domain.x, ylim = domain.y, linewidth = 0, aspectratio = 1)

    x, y = state.u.x[:, 1, 1], state.u.y[1, :, 1]

    u = state.u.data[:, :, 1]
    θ = state.θ.data[:, :, 1]

    ulim = 1
    θlim = 8

    ulevels = range(-ulim, ulim, length = 31)
    θlevels = range(-θlim, θlim, length = 31)

    u_plot = contourf(
        x,
        y,
        clamp.(u, -ulim, ulim)';
        levels = ulevels,
        color = :balance,
        kwargs...,
    )
    θ_plot = contourf(
        x,
        y,
        clamp.(θ, -θlim, θlim)';
        levels = θlevels,
        color = :thermal,
        kwargs...,
    )

    u_title = @sprintf("u at t = %.2f", state.time)
    θ_title = @sprintf("θ at t = %.2f", state.time)

    plot(u_plot, θ_plot, title = [u_title θ_title], size = (600, 250))
end

gif(animation, "shear_instability.mp4", fps = 5)

┌ Info: Plotting frame 1 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 2 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 3 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 4 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 5 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 6 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 7 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 8 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 9 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 10 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 11 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 12 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 13 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 14 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 15 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 16 of 96...
└ @ Main.##597 string:5
┌ Info: Plotting frame 17 of 96...
└ @ Main.##597 string:5
┌ Info

Plots.AnimatedGif("/central/scratch/climaci/climatemachine-docs/1125/climatemachine-docs/docs/src/generated/Ocean/shear_instability.mp4")

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*